<h1>필수패키지 설치</h1>

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


<h1>메타데이터 결합 + 이외 전처리</h1>

In [123]:
import pandas as pd
import numpy as np

###################
### 데이터 리드 ###
###################

# READ CSV
train = pd.read_csv('data_0607_including_petID.csv', index_col=0)

# READ METADATA
Metadata = pd.read_csv('Metadata.csv')
train = pd.merge(train, Metadata, left_on="PetID", right_on="PetID")
#print(train.columns)

### 컬럼 순서 정렬 + 필요없는 컬럼 DROP
columns_list = ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 
               'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
               'Sterilized', 'Health', 'Fee', 'State', 'VideoAmt', 'PhotoAmt', 
               'DesSem', 'DesSemCat', 'NameSem','NameSemCat', 
               'Description_SIZE', 'Description_VALUABLE_SIZE', 'Description_SCORE', 
               'Object_SIZE', 'Object_COLOR_H', 'Object_COLOR_S', 'Object_COLOR_V', 'Image_WIDTH', 'Image_HEIGHT',
               'AdoptionSpeed']
train = train[columns_list]

###################
### 전처리 시작 ###
###################

### AdoptionSpeed Class : 4 -> Binary
### 0 -> Adopt, 1 -> Euthanasia
target_index = 'AdoptionSpeed'
for i in range(train.shape[0]):
    if(train.iloc[i, columns_list.index(target_index)] <= 2):
        train.iloc[i, columns_list.index(target_index)] = 0
    else :
        train.iloc[i, columns_list.index(target_index)] = 1
train[target_index] = pd.to_numeric(train[target_index])

### Age Class : 0~255 -> 1, 2, 3
### 1 -> (0month < Age < 6month), 2 -> (6month < Age < 12month), 3 -> (12month < Age)
target_index = 'Age'
for i in range(train.shape[0]):
    train.iloc[i, columns_list.index(target_index)] = int(train.iloc[i, columns_list.index(target_index)] / 6)
    if(train.iloc[i, columns_list.index(target_index)] > 2):
        train.iloc[i, columns_list.index(target_index)] = 2
train[target_index] = pd.to_numeric(train[target_index])

### Fee Class
target_index = 'Fee'
for i in range(train.shape[0]):
    train.iloc[i, columns_list.index(target_index)] = int(train.iloc[i, columns_list.index(target_index)] / 10)
    if(train.iloc[i, columns_list.index(target_index)] > 25):
        train.iloc[i, columns_list.index(target_index)] = 25
train['Fee'] = pd.to_numeric(train['Fee'])

###################
### 데이터 저장 ###
###################

#train.to_csv("C:/Users/dong_h02/Desktop/기계학습/전처리.csv")
#train.head()

print("Finish!!!")

Finish!!!


 <h1>COLOR 전처리</h1>

In [126]:
# 색상 값에 대한 매핑
color_mapping = {
    1: 'black',
    2: 'brown',
    3: 'golden',
    4: 'yellow',
    5: 'cream',
    6: 'gray',
    7: 'white'
}

# Color1, Color2, Color3에 대한 one-hot 인코딩
for color_col in ['Color1', 'Color2', 'Color3']:
    train[color_col] = train[color_col].map(color_mapping)
    data_encoded = pd.get_dummies(train[color_col], prefix=f'Color_{color_col}')

    # 원래 데이터프레임에 인코딩된 컬럼을 추가
    train = pd.concat([train, data_encoded], axis=1)
#print(train.columns)

###################
### 컬럼   정렬 ###
###################

columns_list = ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color_Color1_black',
                'Color_Color1_brown', 'Color_Color1_cream', 'Color_Color1_golden',
                'Color_Color1_gray', 'Color_Color1_white', 'Color_Color1_yellow',
                'Color_Color2_brown', 'Color_Color2_cream', 'Color_Color2_golden',
                'Color_Color2_gray', 'Color_Color2_white', 'Color_Color2_yellow',
                'Color_Color3_cream', 'Color_Color3_golden', 'Color_Color3_gray',
                'Color_Color3_white', 'Color_Color3_yellow', 
                'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
                'Sterilized', 'Health', 'Fee', 'State', 'VideoAmt', 'PhotoAmt', 
                'DesSem', 'DesSemCat', 'NameSem','NameSemCat', 
                'Description_SIZE', 'Description_VALUABLE_SIZE', 'Description_SCORE', 
                'Object_SIZE', 'Object_COLOR_H', 'Object_COLOR_S', 'Object_COLOR_V', 'Image_WIDTH', 'Image_HEIGHT',
                'AdoptionSpeed']
train = train[columns_list]

#train.head()

print("Finish!!!")

Finish!!!


 <h1>Train Test 분리</h1>

In [129]:
from sklearn.model_selection import train_test_split

#Split Dataset
X, y = train.iloc[:,:-1],train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print("Train :", X_train.shape)
print("Test :", X_test.shape)

Train : (9252, 46)
Test : (2313, 46)


 <h1>XGBOOST</h1>

In [132]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#XGBoost
xg_clf = xgb.XGBClassifier(objective ='reg:squarederror', colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 1000, alpha = 1, n_estimators = 5000)
xg_clf.fit(X_train,y_train)

preds = xg_clf.predict(X_test)

    
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, preds)
print("정확도 : " + str(accuracy*100) + "%")


정확도 : 65.1534803285776%


 <h1>틀린 데이터만 추출</h1>

In [151]:
Miss_list = []
for i in range(X_test.shape[0]):
    if preds[i] != y_test.iloc[i:i+1].values[0]:
        preds_list = []
        for j in range(X_test.shape[1]):
            preds_list.append(X_test.iloc[i:i+1, j].values[0])
        Miss_list.append(preds_list)

Miss_df = pd.DataFrame(Miss_list, columns=[X_test.columns])

Miss_df.head()

,Type,Age,Breed1,Breed2,Gender,Color_Color1_black,Color_Color1_brown,Color_Color1_cream,Color_Color1_golden,Color_Color1_gray,...,NameSemCat,Description_SIZE,Description_VALUABLE_SIZE,Description_SCORE,Object_SIZE,Object_COLOR_H,Object_COLOR_S,Object_COLOR_V,Image_WIDTH,Image_HEIGHT
0,2,0,266,0,2,1,0,0,0,0,...,0,24,7,2.0,9,101,37,134,400,300
1,2,0,266,266,1,0,1,0,0,0,...,1,21,5,0.6,10,17,52,126,640,640
2,2,0,277,260,2,0,0,0,0,1,...,0,174,29,1.4,40,33,130,92,270,480
3,1,0,42,0,2,0,0,0,0,0,...,0,58,12,3.2,88,42,78,139,484,480
4,2,0,266,0,2,1,0,0,0,0,...,0,259,40,7.1,113,58,96,120,373,320


In [153]:
cnt = 0

target_index = 'Type'
        
for i in range(Miss_df.shape[0]):
    if (train.iloc[i, columns_list.index(target_index)] == 1):
        cnt += 1
print(Miss_df.shape[0])
print(cnt)

806
463


 <h1>의사결정나무</h1>

In [131]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mt # 성능지표를 계산하기 위해 import

# 의사결정나무 모델 학습
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# 의사결정나무 모델 평가
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Train_Accuracy : ", dt.score(X_test, y_test), '\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)


dt_pruned = DecisionTreeClassifier(max_depth=5, random_state=156) # max_depth=3으로 제한
dt_pruned.fit(X_train, y_train)

y_pred = dt_pruned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

# print("Accuracy of test set: {:.3f}".format(dt_pruned.score(X_test, y_test)))

Accuracy: 0.5767401642888024
Train_Accuracy :  0.5767401642888024 

Accuracy:  0.58 

Recall:  0.57 

Precision:  0.56 

F1_score:  0.57 

Confusion Matrix: 
 [[693 501]
 [478 641]]
Accuracy: 0.6329442282749675


<h1>메타데이터 추출 + 이미지 READ후 SIZE 추출</h1>
<h5>실행 절대 ㄴ 3시간 걸림</h5>

In [3]:
import pandas as pd
import json
import os
from IPython.display import clear_output

sentiment_dir = 'C:/Users/dong_h02/Desktop/기계학습/petfinder-adoption-prediction/train_sentiment'
metadata_dir = 'C:/Users/dong_h02/Desktop/기계학습/petfinder-adoption-prediction/train_metadata'
image_dir = 'C:/Users/dong_h02/Desktop/기계학습/petfinder-adoption-prediction/train_images'

sentiment_word_cnt_list = []
sentiment_word_score_list = []
sentiment_valuable_word_cnt_list = []

df = pd.read_csv('C:/Users/dong_h02/Desktop/기계학습/petfinder-adoption-prediction/train/train.csv')

# sentiment
for i in range(df['PetID'].size):
    print(i, " Pet ID :", df['PetID'][i] , end = '')
    clear_output(wait = True)
    sentiment_json = sentiment_dir + "/" + df['PetID'][i] + '.json'
    if os.path.isfile(sentiment_json):
        with open(sentiment_json, encoding='UTF8') as f:
            data = json.load(f)
            total_cnt = 0
            total_score = 0
            for j in range(len(data['sentences'])):
                total_cnt = total_cnt + len(data['sentences'][j]['text']['content'].split())
                total_score = total_score + data['sentences'][j]['sentiment']['score']
            sentiment_word_cnt_list.append(total_cnt)
            sentiment_word_score_list.append(total_score)
            sentiment_valuable_word_cnt_list.append(len(data['entities']))
            #print("   Finish!!!")
    else:
        sentiment_word_cnt_list.append(-1)
        sentiment_word_score_list.append(-1)
        sentiment_valuable_word_cnt_list.append(-1)
        #print("   Fail!!!")
        
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")

Finish!!!
Finish!!!
Finish!!!
Finish!!!
Finish!!!


In [4]:
import colorsys

metadata_object_cnt_list = []
metadata_object_h_list = []
metadata_object_s_list = []
metadata_object_v_list = []

# metadata
file_list = os.listdir(metadata_dir)
for i in range(df['PetID'].size):
    print(i, " Pet ID :", df['PetID'][i])
    clear_output(wait = True)
    duplicate_list = []
    total_cnt = 0
    total_h = 0
    total_s = 0
    total_v = 0
    hsv_cnt = 0
    for j in range(len(file_list)):
        if df['PetID'][i] in file_list[j]:
            duplicate_list.append(file_list[j])
    for j in range(len(duplicate_list)):
        metadata_json = metadata_dir + "/" + duplicate_list[j]
        try:
            with open(metadata_json, encoding='UTF8') as f:
                data = json.load(f)
                total_cnt = total_cnt + len(data['labelAnnotations'])
                for n in range(len(data['imagePropertiesAnnotation']['dominantColors']['colors'])):
                    color = data['imagePropertiesAnnotation']['dominantColors']['colors'][n]['color']
                    (r, g, b) = (color['red'] / 255, color['green'] / 255, color['blue'] / 255)
                    (h, s, v) = colorsys.rgb_to_hsv(r, g, b)
                    (h, s, v) = (int(h * 179), int(s * 255), int(v * 255))
                    total_h = total_h + h
                    total_s = total_s + s
                    total_v = total_v + v
                    hsv_cnt = hsv_cnt + 1
        except KeyError:
            continue
                
    metadata_object_cnt_list.append(total_cnt)
    if hsv_cnt == 0:
        metadata_object_h_list.append(-1)
        metadata_object_s_list.append(-1)
        metadata_object_v_list.append(-1)
    else:
        metadata_object_h_list.append(int(total_h / hsv_cnt))
        metadata_object_s_list.append(int(total_s / hsv_cnt))
        metadata_object_v_list.append(int(total_v / hsv_cnt))
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")

Finish!!!
Finish!!!
Finish!!!
Finish!!!
Finish!!!


In [5]:
from PIL import Image

image_h_list = []
image_w_list = []

# image
file_list = os.listdir(image_dir)
for i in range(df['PetID'].size):
    print(i, " Pet ID :", df['PetID'][i])
    clear_output(wait = True)
    duplicate_list = []
    total_cnt = 0
    total_h = 0
    total_w = 0
    for j in range(len(file_list)):
        if df['PetID'][i] in file_list[j]:
            duplicate_list.append(file_list[j])
    for j in range(len(duplicate_list)):
        img_dir = image_dir + "/" + duplicate_list[j]
        im = Image.open(img_dir)
        width, height = im.size
        total_h += height
        total_w += width
        total_cnt += 1
        
    if total_cnt == 0:
        image_h_list.append(-1)
        image_w_list.append(-1)
    else:
        image_h_list.append(int(total_h / total_cnt))
        image_w_list.append(int(total_w / total_cnt))
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")
print("Finish!!!")

Finish!!!
Finish!!!
Finish!!!
Finish!!!
Finish!!!


In [6]:
Metadata_df = pd.DataFrame({'PetID':df['PetID'],
                            'Description_SIZE':sentiment_word_cnt_list,
                            'Description_VALUABLE_SIZE':sentiment_valuable_word_cnt_list,
                            'Description_SCORE':sentiment_word_score_list,
                            'Object_SIZE':metadata_object_cnt_list,
                            'Object_COLOR_H':metadata_object_h_list,
                            'Object_COLOR_S':metadata_object_s_list,
                            'Object_COLOR_V':metadata_object_v_list,
                            'Image_WIDTH':image_w_list,
                            'Image_HEIGHT':image_h_list})
Metadata_df.to_csv("C:/Users/dong_h02/Desktop/기계학습/Metadata.csv")
Metadata_df

,PetID,Description_SIZE,Description_VALUABLE_SIZE,Description_SCORE,Object_SIZE,Object_COLOR_H,Object_COLOR_S,Object_COLOR_V,Image_WIDTH,Image_HEIGHT
0,86e1089a3,69,13,1.8,9,115,88,152,360,480
1,6296e909a,23,2,-0.5,17,38,98,94,349,388
2,3422e4906,69,13,1.4,68,18,68,135,357,342
3,5842f1ff5,25,6,0.9,77,15,75,142,375,325
4,850a43f90,81,13,3.5,29,38,86,155,516,444
...,...,...,...,...,...,...,...,...,...,...
14988,dc0935a84,78,7,2.4,30,35,43,108,360,480
14989,a01ab5b30,31,7,0.3,30,35,108,97,366,333
14990,d981b6395,9,2,2.5,48,40,56,105,360,480
14991,e4da1c9e4,18,4,0.9,30,36,103,83,356,366


<h1>Old 전처리</h1>

In [1]:
### Name 전처리
# name_index = 1
# for i in range(train.shape[0]):
#     if(train.iloc[i, name_index] == "NaN"):
#         train.iloc[i, name_index] = 0
#     else :
#         List = [char for char in str(train.iloc[i, name_index])]
#         cnt = 0;
#         for j in List:
#             if j.isalnum():
#                 cnt += 1;
#         train.iloc[i, name_index] = len(str(train.iloc[i, name_index])) - cnt
            
#         #train.iloc[i, 2] = len(str(train.iloc[i, 2]))
#     #print(train.iloc[i, 2])
# train['Name'] = pd.to_numeric(train['Name'])

### Breed 전처리
# name_index = 2
# for i in range(train.shape[0]):
#     if (train.iloc[i, name_index] != 307 and train.iloc[i, name_index] != 0) and (train.iloc[i, name_index + 1] != 307 and train.iloc[i, name_index + 1]):
#         train.iloc[i, name_index] = 0
#     elif (train.iloc[i, name_index] != 307 and train.iloc[i, name_index] != 0) or (train.iloc[i, name_index + 1] != 307 and train.iloc[i, name_index + 1]):
#         train.iloc[i, name_index] = 1
#     else:
#         train.iloc[i, name_index] = 2
# train['Breed1'] = pd.to_numeric(train['Breed1'])

### State 전처리
# name_index = 15
# state_index = 0
# Statelist = []
# for i in range(train.shape[0]):
#     if train.iloc[i, name_index] in Statelist:
#         #print(train.iloc[i, name_index], end ='')
#         train.iloc[i, name_index] = Statelist.index(train.iloc[i, name_index])
#         #print(train.iloc[i, name_index])
#     else:
#         #print(train.iloc[i, name_index], end ='')
#         Statelist.append(train.iloc[i, name_index])
#         train.iloc[i, name_index] = state_index
#         state_index += 10
#         #print(train.iloc[i, name_index])
# train['State'] = pd.to_numeric(train['State'])

### Drop
#Remove duplication
#train.drop(['Unnamed: 0.1'],axis = 1, inplace = True)
# train.drop(['Name'],axis = 1, inplace = True)